In [34]:
from Greeks_cal import *
from get_trade_signal import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [35]:
# option_df
series = pd.read_csv(r".\series.csv")
series = series[series["合约系列"] != "小计"]
series["合约系列"] = series["合约系列"].apply(lambda x: x.strip())
series = series.loc[series.groupby("trade_date")["持仓量"].idxmax(), :].set_index(["trade_date", "合约系列"])

contracts = pd.read_csv(r".\data.csv")
contracts["合约系列"] = contracts["合约代码"].apply(lambda x: x[:6])
contracts["trade_date"] = pd.to_datetime(contracts["trade_date"])
contracts = contracts.set_index(["trade_date", "合约系列"]).loc[series.index, :].reset_index()
contracts["C/P"] = contracts["合约代码"].apply(lambda x: x[6])
contracts["trade_date"] = pd.to_datetime(contracts["trade_date"])
contracts = contracts.loc[contracts.groupby([contracts["trade_date"], contracts["C/P"]])["成交量"].idxmax(), :].set_index(["trade_date", "C/P"])

spot_price = pd.read_csv(r".\with_spot_price.csv")

updated_contracts = calculate_greeks(contracts, series, spot_price)
updated_contracts = updated_contracts.reset_index()
updated_contracts = updated_contracts[['合约代码', 'trade_date', '收盘价', '成交量', '持仓量', 'delta', 'gamma', 'theta', 'vega', 'rho']]
option_df = updated_contracts.rename(columns={'合约代码': 'id', 'trade_date': 'time', '收盘价': 'close', '成交量': 'volume', '持仓量': 'open_interest'})
option_df.head(5)

,id,time,close,volume,open_interest,delta,gamma,theta,vega,rho
0,au2412C640,2024-11-01,8.90,8214,2152,0.463252,0.006721,-84.789120,96.235688,40.234454
1,au2412P600,2024-11-01,4.22,11958,5444,-0.275755,0.005653,-70.425104,80.942961,27.255401
2,au2412C640,2024-11-04,6.62,7715,2035,0.422374,0.006674,-87.055269,91.258671,34.419134
3,au2412P600,2024-11-04,4.30,10193,4926,-0.311821,0.006031,-78.360607,82.473768,28.909370
4,au2412C640,2024-11-05,7.08,7594,2253,0.414356,0.006837,-85.915842,89.879945,33.145930


In [36]:
# futures_df
spot_price = pd.read_csv(r".\with_spot_price.csv")
spot_price = spot_price[['dominant_contract', 'trade_date', 'dominant_contract_price']]
futures_df = spot_price.rename(columns={'dominant_contract': 'id', 'trade_date': 'time', 'dominant_contract_price': 'close'})
futures_df['time'] = pd.to_datetime(futures_df['time'].astype(str), format='%Y%m%d')
futures_df = futures_df.drop_duplicates().reset_index(drop=True)
futures_df = futures_df.sort_values('time')
futures_df = futures_df.reset_index(drop=True)
futures_df.head(5)

,id,time,close
0,au2304,2023-01-03,410.98
1,au2304,2023-01-04,412.62
2,au2304,2023-01-05,414.54
3,au2304,2023-01-06,410.38
4,au2304,2023-01-09,412.28


In [37]:
# paras
paras = pd.DataFrame({
    'range': [[-np.inf, -50], [-50, 0], [0, 50], [50, np.inf]],
    'delta': [[-0.1, 0.1], [0.4, 0.6], [0.9, 1.1], [-0.1, 0.1]],
    'gamma': [[-0.1, 0.1], [-0.1, 0.1], [-0.1, 0.1], [-0.1, 0.1]],
    'theta': [[-0.1, 0.1], [-0.1, 0.1], [-0.1, 0.1], [-0.1, 0.1]],
    'vega': [[-0.1, 0.1], [-0.1, 0.1], [-0.1, 0.1], [-0.1, 0.1]],
    'rho': [[-0.1, 0.1], [-0.1, 0.1], [-0.1, 0.1], [-0.1, 0.1]]
})
start_time = pd.to_datetime('2024-11-01')

"""paras应调整为资金希腊值。设资金量为equity，策略起始日黄金价格为spot0，合约乘数为multiplier，那么每个希腊值都应乘上equity/(spot0*multiplier)。"""
equity = 10000000
multiplier = 1000
spot0 = futures_df[futures_df['time'] < start_time]['close'].iloc[-1]
paras[['delta', 'gamma', 'theta', 'vega', 'rho']] = paras[['delta', 'gamma', 'theta', 'vega', 'rho']].apply(lambda x: x.apply(lambda y: [z * equity/(spot0*multiplier) for z in y]))
paras

,range,delta,gamma,theta,vega,rho
0,"[-inf, -50]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]"
1,"[-50, 0]","[6.283972727558362, 9.425959091337544]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]"
2,"[0, 50]","[14.138938637006316, 17.280925000785498]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]"
3,"[50, inf]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]","[-1.5709931818895906, 1.5709931818895906]"


In [38]:
position_option, position_futures = get_trading_signal(option_df, futures_df, paras, start_time)


Time: 2024-11-01 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 9.0]

Time: 2024-11-04 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 9.0]

Time: 2024-11-05 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 9.0]

Time: 2024-11-06 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 9.0]

Time: 2024-11-07 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 7.0]

Time: 2024-11-08 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 7.0]

Time: 2024-11-11 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 7.0]

Time: 2024-11-12 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 7.0]

Time: 2024-11-13 00:00:00
Status: Optimal
Optimal value is: 0.0
Solution is: [0.0, 0.0, 7.0]

Time: 2024-11-14 00:00:00
Status: Optimal
Optimal value is: -7.02
Solution is: [3.0, -3.0, 5.0]

Time: 2024-11-15 00:00:00
Status: Optimal
Optimal value 

In [39]:
position_option

,id,time,position
0,au2412C640,2024-11-01,0.0
1,au2412P600,2024-11-01,0.0
2,au2412C640,2024-11-04,0.0
3,au2412P600,2024-11-04,0.0
4,au2412C640,2024-11-05,0.0
5,au2412P600,2024-11-05,0.0
6,au2412C640,2024-11-06,0.0
7,au2412P600,2024-11-06,0.0
8,au2412C640,2024-11-07,0.0
9,au2412P600,2024-11-07,0.0


In [40]:
position_futures

,id,time,position
0,au2412,2024-11-01,9.0
1,au2502,2024-11-04,9.0
2,au2502,2024-11-05,9.0
3,au2502,2024-11-06,9.0
4,au2502,2024-11-07,7.0
5,au2502,2024-11-08,7.0
6,au2502,2024-11-11,7.0
7,au2502,2024-11-12,7.0
8,au2502,2024-11-13,7.0
9,au2502,2024-11-14,5.0
